In [56]:
import matplotlib
%matplotlib inline

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array


from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
import tensorflow as tf

In [3]:
EPOCHS = 50
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

In [57]:
class MyModel:
    @staticmethod
    def build_type_branch(inputs, numTypes,
        finalAct="softmax", chanDim=-1):
       
        
      
        x = Conv2D(32, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
       
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(256)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numTypes)(x)
        x = Activation(finalAct, name="type_output")(x)
       
        return x
    
    @staticmethod
    def build_color_branch(inputs, numColors, finalAct="softmax",
        chanDim=-1):
        # CONV => RELU => POOL
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        # CONV => RELU => POOL
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        # CONV => RELU => POOL
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numColors)(x)
        x = Activation(finalAct, name="color_output")(x)
       
        return x
    
    @staticmethod
    def build_company_branch(inputs, numCompanies, finalAct="softmax",
        chanDim=-1):
       
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.5)(x)
       
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.5)(x)
        
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.5)(x)
        
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.5)(x)
        
        
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numCompanies)(x)
        x = Activation(finalAct, name="company_output")(x)
       
        return x
    
    @staticmethod
    def build_view_branch(inputs, numViews, finalAct="softmax",
        chanDim=-1):
       
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
       
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numViews)(x)
        x = Activation(finalAct, name="view_output")(x)
       
        return x
    @staticmethod
    def build_height_branch(inputs, numHeight=1, finalAct="relu",
        chanDim=-1):
        x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)
      
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
       
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(256)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numHeight)(x)
        x = Activation(finalAct, name="height_output")(x)
        
        return x
    
    @staticmethod
    def build_scratch_branch(inputs, numScratch=1, finalAct="sigmoid",
        chanDim=-1):
        x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)
      
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
       
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(256)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numScratch)(x)
        x = Activation(finalAct, name="scratch_output")(x)
        return x
    
    
    
    @staticmethod
    def build_width_branch(inputs, numWidth=1, finalAct="relu",
        chanDim=-1):
        x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)
      
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(64, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
       
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Conv2D(128, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(256)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(numWidth)(x)
        x = Activation(finalAct, name="width_output")(x)
        
        return x
    @staticmethod
    def build(width, height, numTypes, numColors,numCompanies,numViews =5,finalAct="softmax"):
        
        inputShape = (height, width, 3)
        chanDim = -1
        
        inputs = Input(shape=inputShape)
        typeBranch = MyModel.build_type_branch(inputs,
            numTypes, finalAct=finalAct, chanDim=chanDim
        ) 
        colorBranch = MyModel.build_color_branch(inputs,
            numColors, finalAct=finalAct, chanDim=chanDim)
        companyBranch = MyModel.build_company_branch(
            inputs,
            numCompanies, finalAct=finalAct, chanDim=chanDim
        )
        viewBranch =  MyModel. build_view_branch(inputs,
            numViews, finalAct=finalAct, chanDim=chanDim
        )
        scratchBranch = MyModel.build_scratch_branch(inputs)
        
        model = Model(
            inputs=inputs,
            outputs=[typeBranch, colorBranch
                     #,companyBranch,
                     ##scratchBranch,viewBranch
                    ],
            name="mymodel")
       
        return model
    
       
    
    

In [58]:
EPOCHS = 50
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

In [6]:
losses = {
    "type_output": "categorical_crossentropy",
    "color_output": "categorical_crossentropy",
    "company_output":"categorical_crossentropy",
    #"scratch_output":"binary_crossentropy",
    #"view_output":"categorical_crossentropy"
}

lossWeights = {"type_output": 1.0, "color_output": 1.0,"company_output":1.0,
              #"scratch_output":0.1,"view_output":1.0
              }
model = MyModel.build(96,96,3,10,10)

print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt, loss=losses, loss_weights=lossWeights,
    metrics=["accuracy"])

[INFO] compiling model...


In [7]:
# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images("data")))
random.seed(42)
random.shuffle(imagePaths)
# initialize the data, clothing category labels (i.e., shirts, jeans,
# dresses, etc.) along with the color labels (i.e., red, blue, etc.)
data = []
categoryLabels = []
colorLabels = []

[INFO] loading images...


In [8]:
def augmented_images_generations(path,no_of_images):
    d = os.listdir(path)
     
    for j in range(len(d)):
        (type,color,company,height,width,scratch,view) = d[j].split('_')[:]
        view = view.split('.')[0]
        (type,color,company,height,width,scratch,view) = [x.lower() for x in (type,color,company,height,width,scratch,view)]
        print(type,color,company,height,width,scratch,view)
        img = load_img(os.path.join(path,d[j]))
       
        
        data = img_to_array(img)
        samples = expand_dims(data, 0)
        datagen = ImageDataGenerator(height_shift_range=0.5,zoom_range=0.2,rotation_range=45)
        it = datagen.flow(samples, batch_size=1)
        counter = 1
        os.mkdir(os.path.join("output",type+"_"+color+"_"+company+"_"+scratch+"_"+view))
        for i in range(no_of_images):
            
            batch = it.next()
            image = batch[0].astype('uint8')
            p = os.path.join("output",type+"_"+color+"_"+company+"_"+scratch+"_"+view,str(counter)+".png")
            print(p)
            cv2.imwrite(p,image)
            counter+=1
    
    

In [9]:
augmented_images_generations("data",20)

trolley violet alpha 1032 581 0 right


FileNotFoundError: [Errno 2] No such file or directory: 'output/trolley_violet_alpha_0_right'

In [81]:
os.listdir("output")

['handbag_red_joy_0_back',
 'backpack_grey_wildcraft_0_left',
 'trolley_red_safari_0_left',
 'baggage_blue_x_0_front',
 'suitcase_grey_alpha_0_front',
 '.DS_Store',
 'trolley_violet_alpha_0_left',
 'trolley_violet_alpha_0_right',
 'baggage_red_flightway_0_front',
 'backpack_grey_wildcraft_0_back',
 'trolley_violet_american tourister_0_left',
 'handbag_red_joy_0_left',
 'handbag_red_joy_0_front',
 'backpack_black_hp_0_front',
 'backpack_grey_wildcraft_0_front',
 'backpack_black_dell_0_top',
 'trolley_red_safari_0_right',
 'baggage_red_y_1_front',
 'trolley_violet_alpha_0_back',
 'baggage_red_flightway_0_top',
 'backpack_black_hp_0_right',
 'baggage_red_flightway_0_back',
 'baggage_blue_x_0_back',
 'backpack_black_wildcraft_0_back',
 'trolley_red_safari_0_front',
 'baggage_red_y_1_right',
 'backpack_black_hp_0_left',
 'backpack_blue_hp_1_back',
 'backpack_orange+black_hashtag_0_front',
 'backpack_black_dell_0_back',
 'baggage_red_y_1_left',
 'baggage_red_flightway_0_left',
 'trolley_viol

In [131]:
#Image augmentation
type_labels=[]
color_labels=[]
view_labels=[]
company_labels=[]
scratch_labels=[]
data=[]

for i in os.listdir("output"):
    
    print(i.split('_')[:])
    if(i==".DS_Store"):
        continue
    (type,color,company,scratch,view) = i.split('_')[:]
    view = view.split('.')[0]
    for j in os.listdir("output/"+i):
        image = cv2.imread("output/"+i+"/"+j)
        image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = img_to_array(image)
        data.append(image)
        type_labels.append(type)
        color_labels.append(color)
        view_labels.append(view)
        company_labels.append(company)
        scratch_labels.append(int(scratch))
        
        
        

['handbag', 'red', 'joy', '0', 'back']
['backpack', 'grey', 'wildcraft', '0', 'left']
['trolley', 'red', 'safari', '0', 'left']
['baggage', 'blue', 'x', '0', 'front']
['suitcase', 'grey', 'alpha', '0', 'front']
['.DS', 'Store']
['trolley', 'violet', 'alpha', '0', 'left']
['trolley', 'violet', 'alpha', '0', 'right']
['baggage', 'red', 'flightway', '0', 'front']
['backpack', 'grey', 'wildcraft', '0', 'back']
['trolley', 'violet', 'american tourister', '0', 'left']
['handbag', 'red', 'joy', '0', 'left']
['handbag', 'red', 'joy', '0', 'front']
['backpack', 'black', 'hp', '0', 'front']
['backpack', 'grey', 'wildcraft', '0', 'front']
['backpack', 'black', 'dell', '0', 'top']
['trolley', 'red', 'safari', '0', 'right']
['baggage', 'red', 'y', '1', 'front']
['trolley', 'violet', 'alpha', '0', 'back']
['baggage', 'red', 'flightway', '0', 'top']
['backpack', 'black', 'hp', '0', 'right']
['baggage', 'red', 'flightway', '0', 'back']
['baggage', 'blue', 'x', '0', 'back']
['backpack', 'black', 'wildc

In [39]:
type_labels=[]
color_labels=[]
view_labels=[]
company_labels=[]
scratch_labels=[]
data=[]

for i in os.listdir("data"):
    
    print(i.split('_')[:])
    if(i==".DS_Store"):
        continue
    (type,color,company,height,width,scratch,view) = i.split('_')[:]
    
        
    view = view.split('.')[0]
   
    image = cv2.imread("data/"+i)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = img_to_array(image)
    data.append(image)
    type_labels.append(type)
    color_labels.append(color)
    view_labels.append(view)
    company_labels.append(company)
    scratch_labels.append(int(scratch))
        

['trolley', 'violet', 'alpha', '1032', '581', '0', 'right.jpeg']
['baggage', 'red', 'y', '14', '15', '1', 'left.jpg']
['trolley', 'red', 'safari', '8', '19', '0', 'top.jpg']
['baggage', 'blue', 'x', '12', '13', '0', 'front.jpg']
['baggage', 'blue', 'x', '12', '13', '0', 'right.jpg']
['handbag', 'red', 'joy', '846', '2109', '0', 'left.jpg']
['suitcase', 'grey', 'alpha', '1107', '1967', '0', 'top.jpg']
['.DS', 'Store']
['backpack', 'black', 'dell', '917', '737', '0', 'front.jpg']
['backpack', 'black', 'dell', '861', '1095', '0', 'top.jpg']
['baggage', 'red', 'y', '14', '15', '1', 'top.jpg']
['backpack', 'black', 'hp', '975', '1280', '0', 'left.jpeg']
['backpack', 'grey', 'wildcraft', '1334', '1555', '0', 'front.jpg']
['backpack', 'black', 'dell', '671', '692', '0', 'back.jpg']
['backpack', 'black', 'wildcraft', '780', '1040', '0', 'back.jpg']
['baggage', 'red', 'flightway', '921', '866', '0', 'left.jpg']
['backpack', 'blue', 'hp', '214', '284', '1', 'back.png']
['trolley', 'violet', 'alp

In [40]:
data = np.array(data, dtype="float") / 255.0
print("[INFO] data matrix: {} images ({:.2f}MB)".format(
    len(imagePaths), data.nbytes / (1024 * 1000.0)))

type_labels = np.array(type_labels)
color_labels = np.array(color_labels)
view_labels = np.array(view_labels)
company_labels = np.array(company_labels)
scratch_labels = [int(x) for x in scratch_labels]
scratch_labels = np.array(scratch_labels)



[INFO] data matrix: 40 images (8.64MB)


In [24]:
print(company_labels)

['alpha', 'y', 'safari', 'x', 'x', 'joy', 'alpha', 'dell', 'dell', 'y', 'hp', 'wildcraft', 'dell', 'wildcraft', 'flightway', 'hp', 'alpha', 'y', 'alpha', 'hp', 'joy', 'wildcraft', 'alpha', 'safari', 'safari', 'alpha', 'flightway', 'flightway', 'wildcraft', 'joy', 'safari', 'x', 'x', 'flightway', 'hp', 'hp', 'wildcraft', 'alpha', 'y', 'y']


In [41]:
print("[INFO] binarizing labels...")
typeLB = LabelBinarizer()
colorLB = LabelBinarizer()
viewLB = LabelBinarizer()
companyLB = LabelBinarizer()

type_labels = typeLB.fit_transform(type_labels)
color_labels = colorLB.fit_transform(color_labels)
view_labels = viewLB.fit_transform(view_labels)
company_labels = companyLB.fit_transform(company_labels)


split = train_test_split(data, type_labels, color_labels,view_labels,company_labels,scratch_labels,
    test_size=0.2, random_state=42)

(trainX, testX, trainTypeY, testTypeY,
    trainColorY, testColorY,trainViewY,testViewY,trainCompanyY,testCompanyY,trainScratchY,testScratchY) = split





[INFO] binarizing labels...


In [42]:
print(companyLB.classes_)

['alpha' 'dell' 'flightway' 'hp' 'joy' 'safari' 'wildcraft' 'x' 'y']


In [60]:
losses = {
    "type_output": "categorical_crossentropy",
    "color_output": "categorical_crossentropy",
    #"company_output":"categorical_crossentropy",
    #"scratch_output":"binary_crossentropy",
    #"view_output":"categorical_crossentropy"
}

lossWeights = {"type_output": 1.0, "color_output": 1.0,
               #"company_output":1.0,
              #"scratch_output":0.1,"view_output":1.0
              }
model = MyModel.build(96,96,len(typeLB.classes_),len(colorLB.classes_),len(companyLB.classes_),len(viewLB.classes_))

print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt, loss=losses, loss_weights=lossWeights,
    metrics=["accuracy"],run_eagerly=True)

[INFO] compiling model...


In [61]:
EPOCHS = 10

H = model.fit(x=data,
    y={"type_output": type_labels, "color_output": color_labels
       #,"company_output":company_labels,
       #"scratch_output":scratch_labels,
      #"view_output":view_labels
      },
    epochs=EPOCHS,
    verbose=1)

Epoch 1/10
2/2 [==============================] - 0s 183ms/step - loss: 5.2934 - type_output_loss: 2.4775 - color_output_loss: 2.8159 - type_output_accuracy: 0.2500 - color_output_accuracy: 0.2500
Epoch 2/10
2/2 [==============================] - 0s 150ms/step - loss: 4.3363 - type_output_loss: 2.7127 - color_output_loss: 1.6236 - type_output_accuracy: 0.1500 - color_output_accuracy: 0.4750
Epoch 3/10
2/2 [==============================] - 0s 167ms/step - loss: 3.2002 - type_output_loss: 2.3756 - color_output_loss: 0.8246 - type_output_accuracy: 0.3500 - color_output_accuracy: 0.6500
Epoch 4/10
2/2 [==============================] - 0s 148ms/step - loss: 3.4400 - type_output_loss: 2.6630 - color_output_loss: 0.7770 - type_output_accuracy: 0.2250 - color_output_accuracy: 0.7000
Epoch 5/10
2/2 [==============================] - 0s 172ms/step - loss: 3.2510 - type_output_loss: 2.3694 - color_output_loss: 0.8816 - type_output_accuracy: 0.2750 - color_output_accuracy: 0.6750
Epoch 6/10
2/2 

In [62]:
for i in os.listdir("data"):
    
    image = cv2.imread("data/"+i)
    if(i==".DS_Store"):
        continue
    print(i,end="")
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = img_to_array(image)
    samples = expand_dims(image, 0)
    res = model.predict(samples)

    print(typeLB.classes_[np.argmax(res[0][0])],colorLB.classes_[np.argmax(res[1][0])])

trolley_violet_alpha_1032_581_0_right.jpegtrolley violet
baggage_red_y_14_15_1_left.jpgtrolley grey
trolley_red_safari_8_19_0_top.jpgtrolley violet
baggage_blue_x_12_13_0_front.jpgtrolley grey
baggage_blue_x_12_13_0_right.jpgtrolley violet
handbag_red_joy_846_2109_0_left.jpgtrolley grey
suitcase_grey_alpha_1107_1967_0_top.jpgtrolley grey
backpack_black_dell_917_737_0_front.jpgtrolley violet
backpack_black_dell_861_1095_0_top.jpgtrolley violet
baggage_red_y_14_15_1_top.jpgtrolley violet
backpack_black_hp_975_1280_0_left.jpegtrolley violet
backpack_grey_wildcraft_1334_1555_0_front.jpgtrolley violet
backpack_black_dell_671_692_0_back.jpgtrolley violet
backpack_black_wildcraft_780_1040_0_back.jpgtrolley violet
baggage_red_flightway_921_866_0_left.jpgtrolley grey
backpack_blue_hp_214_284_1_back.pngtrolley violet
trolley_violet_alpha_1032_581_0_left.jpegtrolley violet
baggage_red_y_14_15_1_back.jpgtrolley violet
suitcase_grey_alpha_751_1288_0_left.jpgtrolley violet
backpack_black_hp_975_1280

In [51]:
model.save("model1.h5")

In [32]:
print(typeLB.classes_)

['backpack' 'baggage' 'handbag' 'suitcase' 'trolley']


In [33]:
print(companyLB.classes_)

['alpha' 'dell' 'flightway' 'hp' 'joy' 'safari' 'wildcraft' 'x' 'y']


In [52]:
f = open("type_lb.pickle","wb")


f.write(pickle.dumps(typeLB))

f = open("color_lb.pickle","wb")

f.write(pickle.dumps(colorLB))

f = open("company_lb.pickle","wb")

f.write(pickle.dumps(companyLB))


print("HI")

HI


In [22]:
f =  open("type_lb.pickle","rb") 
t=pickle.loads(f.read())

In [236]:
t.classes_

array(['backpack', 'baggage', 'handbag', 'suitcase', 'trolley'],
      dtype='<U8')

In [239]:
def fun():
    return {'path':'123',
    'type':typeLB.classes_[np.argmax(res[0][0])],
    'color':colorLB.classes_[np.argmax(res[1][0])],
    'company':companyLB.classes_[np.argmax(res[2][0])]
    }

In [63]:

import tensorflow as tf

from keras.models import model_from_json
model.save('model2.model',save_format="h5")

from keras.models import load_model

x=load_model('model2.model',custom_objects={"": tf})

In [64]:
for i in os.listdir("data"):
    
    image = cv2.imread("data/"+i)
    if(i==".DS_Store"):
        continue
    print(i,end="")
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = img_to_array(image)
    samples = expand_dims(image, 0)
    res = x.predict(samples)

    print(typeLB.classes_[np.argmax(res[0][0])],colorLB.classes_[np.argmax(res[1][0])])

trolley_violet_alpha_1032_581_0_right.jpegtrolley violet
baggage_red_y_14_15_1_left.jpgtrolley grey
trolley_red_safari_8_19_0_top.jpgtrolley violet
baggage_blue_x_12_13_0_front.jpgtrolley grey
baggage_blue_x_12_13_0_right.jpgtrolley violet
handbag_red_joy_846_2109_0_left.jpgtrolley grey
suitcase_grey_alpha_1107_1967_0_top.jpgtrolley grey
backpack_black_dell_917_737_0_front.jpgtrolley violet
backpack_black_dell_861_1095_0_top.jpgtrolley violet
baggage_red_y_14_15_1_top.jpgtrolley violet
backpack_black_hp_975_1280_0_left.jpegtrolley violet
backpack_grey_wildcraft_1334_1555_0_front.jpgtrolley violet
backpack_black_dell_671_692_0_back.jpgtrolley violet
backpack_black_wildcraft_780_1040_0_back.jpgtrolley violet
baggage_red_flightway_921_866_0_left.jpgtrolley grey
backpack_blue_hp_214_284_1_back.pngtrolley violet
trolley_violet_alpha_1032_581_0_left.jpegtrolley violet
baggage_red_y_14_15_1_back.jpgtrolley violet
suitcase_grey_alpha_751_1288_0_left.jpgtrolley violet
backpack_black_hp_975_1280

In [66]:
from flask import request
from flask import jsonify
from flask_cors import CORS, cross_origin
from flask import render_template
from flask import Flask
import base64

In [67]:
app = Flask(__name__)

In [68]:
tem = [{'path': 'SIH_Video/7/i5_1.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i6_1.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i6_2.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i6_3.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i8_1.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i8_2.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i9_1.png', 'type': 'trolley', 'color': 'grey'}, {'path': 'SIH_Video/7/i10_1.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i11_1.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i12_1.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i12_2.png', 'type': 'trolley', 'color': 'violet'}, {'path': 'SIH_Video/7/i13_1.png', 'type': 'trolley', 'color': 'grey'}, {'path': 'SIH_Video/7/i15_1.png', 'type': 'trolley', 'color': 'grey'}]

In [70]:
for i in tem:
    print(i['color'])

violet
violet
violet
violet
violet
violet
grey
violet
violet
violet
violet
grey
grey
